In [ ]:
# Esta es una modificacion del script original para google colab, para que funcione en github codespaces
# Hecho por @LUCAST o @Lucast1574 o @Lucast_1574 en github y otras redes sociales
# Versiones disponibles testeadas:
# - 1.20.4
# - 1.20.3
# - 1.20.2
# - 1.20.1
# - 1.20
# - 1.19.4
# - 1.19.3
# - 1.19.2
# - 1.19.1
# - 1.19
# - 1.18.2
# - 1.18.1
# - 1.18
# - 1.17.1
# - 1.17
# - 1.16.5
# - 1.16.4
# - 1.16.3
# - 1.15.2
# - 1.14.4
# - 1.13.2
# - 1.12.2
# - 1.11.2
# - 1.10.2
# - 1.9.4
# - 1.8.8


version = '1.20.1' # Version de Minecraft a usar

#Chose server type. Currently available versions:
# - paper (Lo más vanilla. Es lo mismo que SPIGOT, bukkit, etc)
# - forge (Para usar MODS. Es básicamente FORGE) [Aviso, la instalación tarda APROX 10-15 min, al ejecutar el primer paso, esperar a que ponga COMPLETADO]
# - fabric (Para jugar Fabric. Es una alternativa a Forge)
# - vanilla (Minecraft Vanilla sin mods ni plugins)
# - snapshot (En "version" tendrás que poner el nombre de la SNAPSHOT, EJ: 23w42a)
# - mohist (Versión de Minecraft que combina Forge y Bukkit)

server_type = 'forge'

#Importa librerias necesarias para el funcionamiento del script

import os
import requests
import json


server_directory = "Minecraft-server"

# Crea el directorio del servidor en codespace

os.makedirs(server_directory, exist_ok=True)
os.chdir(server_directory)

# Inicio interno... Descarga el servidor

if server_type == 'paper':
    a = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version)

    b = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]))

    print("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"])
    serverURL = "https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"]

if server_type == 'forge':
    serverURL = "https://serverjars.com/api/fetchJar/modded/forge/" + version

if server_type == 'vanilla':
    serverURL = "https://serverjars.com/api/fetchJar/vanilla/vanilla/" + version

if server_type == 'snapshot':
    serverURL = "https://serverjars.com/api/fetchJar/vanilla/snapshot/" + version
if server_type == 'fabric':
    serverURL = 'https://maven.fabricmc.net/net/fabricmc/fabric-installer/0.11.2/fabric-installer-0.11.2.jar'

if server_type == 'mohist':
    serverURL = "https://serverjars.com/api/fetchJar/modded/mohist/" + version

jar_name = {'paper': 'server.jar', 'fabric': 'fabric-installer.jar', 'forge': 'forge.jar', 'vanilla': 'vanilla.jar', 'snapshot': "snapshot.jar", 'mohist': "mohist.jar"}

print('Descargando el tipo de servidor: ')

if serverURL:
    print('Descargando a GitHub Codespaces...')
    r = requests.get(serverURL)

    if r.status_code == 200:
        with open(jar_name[server_type], 'wb') as f:
            f.write(r.content)
    else:
        print('Error '+ str(r.status_code) + '! La versión que has elegido probablemente no funciona.')


# Correr en un camino especifico

if server_type == 'fabric':
    !java -jar fabric-installer.jar server -mcversion $version -downloadMinecraft

if server_type == 'forge':
    %cd "/workspaces/codespaces-jupyter/Minecraft-server"
    !java -jar forge.jar --installServer

# Guarda el archivo de configuración localmente
colabconfig = {"server_type": server_type, "server_version": version}
with open("colabconfig.json", 'w') as config_file:
    json.dump(colabconfig, config_file)

# Imprime un mensaje de finalización
print('Completado!')
# Cambie el eula a true para aceptar los términos de Minecraft
!echo "eula=true" >> eula.txt


In [1]:
#importa librerias necesarias para ejecucion

import requests

import time

import os

import re

import json

import glob

import threading



#Actualizacion de lista de paquetes

!sudo apt update &>/dev/null && echo "el apt cache fue actualizado correctamente" || echo "la actualizacion del apt cache fallo, es posible que recibas paquetes obsoletos"

#Asignar archivos al directorio


os.makedirs("/workspaces/codespaces-jupyter/Minecraft-server", exist_ok=True)

os.chdir("/workspaces/codespaces-jupyter/Minecraft-server")

!ls #lista los archivos del directorio para verificarlos



#Importacion del archivo de configuracion


config_path = "/workspaces/codespaces-jupyter/Minecraft-server/colabconfig.json"

if os.path.isfile(config_path):

    with open(config_path) as config_file:

        colabconfig = json.load(config_file)

else:

    colabconfig = {"server_type": "generic"} # Usar la configura default si no existe

    with open(config_path, 'w') as new_config_file:

        json.dump(colabconfig, new_config_file)



# Instalar OpenJDK


version = colabconfig["server_version"]

if colabconfig["server_version"] < "1.17":

    !sudo apt-get purge openjdk* > /dev/null 2>&1

    !sudo apt-get install openjdk-8-jre-headless &>/dev/null and print("OpenJDK 8 instalado!") or print("Error al intalar OpenJDK 8.")

else:

    !sudo apt-get purge openjdk* > /dev/null 2>&1

    !sudo apt-get install openjdk-17-jre-headless &>/dev/null and print("OpenJDK 17 instalado!") or print("Error al intalar OpenJDK 17.")


# Chequeo de version de java

java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'

if java_ver[0] == "17":

    print("Estas usando JAVA 17.")

else:

    print("Estas usando una version desactualizada de java. Puedo que Minecraft 1.17+ no funcione.")



#Lista de posibles archivos .jar


jar_list = {'paper': 'server.jar', 'fabric': 'fabric-server-launch.jar', 'generic': 'server.jar', 'forge': 'forge.jar', 'vanilla': 'vanilla.jar', 'snapshot': 'snapshot.jar', 'mohist': "mohist.jar"}

jar_name = jar_list[colabconfig["server_type"]]



# Java arguments.

if colabconfig["server_type"] == "paper":

    server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"

else:

    server_flags = "" # No se necesitan banderas para servidores que no sean de paper

#Argumentos JVM

memory_allocation = "-Xms10G -Xmx10G"



# Elige el servicio de tunelacion que desees

# Opciones disponibles: ngrok

tunnel_service = "ngrok"

print("Usando", tunnel_service)


#Condicional de tunelacion del servidor



if tunnel_service == "ngrok":

    !pip -q install pyngrok #Aqui se instala el ngrok en el servidor

    from pyngrok import conf, ngrok



    # Obten tu token de ngrok

    print("Consigue tu authtoken de https://dashboard.ngrok.com/auth") #Consigue aqui tu token de ngrok, sera dinamico, si quieres que sea estatico, necesitaras la version premiun de ngrok

    import getpass



    # v - - - - - - - TOKEN - - - - - - - v



    authtoken = "tu token aqui" # <---- TOKEN NGROK (PON AQUÍ TU TOKEN [TIENE QUE ESTAR ENTRE LAS "COMILLAS"])

    ! ngrok authtoken $authtoken # logeo en ngrok



    #Establecimiento de region de ngrok



    #v - - - - - - - REGIONES DISPONIBLES - - - - - - - v

    # ap - Asia/Pacifico (Singapore)

    # au - Australia (Sydney)

    # eu - Europa (Frankfurt - Alemania)

    # in - India (Mumbai)

    # jp - Japon (Tokyo)

    # sa - America del sur (São Paulo - Brasil)

    # us - Estados unidos (Ohio)

    conf.get_default().region = 'us' # <--- Cambia esto por la region que quieras



    # Conectar a ngrok

    url = ngrok.connect(25565, 'tcp')

    print('La IP de tu servidor es ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))

    #Si obtienes el servicio premiun de ngrok borra las dos lineas de codigo de arriba y quitale el # a estas a continuacion:

    #subdominio = "mi-subdominio"

    #url = ngrok.connect(25565, 'tcp', subdomain=subdominio)

    #print('La IP de tu servidor es ' + url.replace('tcp://', ''))



print('Iniciando servidor...')

def get_users():
    url = 'https://jsonplaceholder.typicode.com/users'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            print("Lista de usuarios:")
            for user in data:
                print(f"Nombre: {user['name']}, Email: {user['email']}")
        else:
            print(f"Error al hacer la solicitud. Código de estado: {response.status_code}")
    except requests.RequestException as e:
        print(f"Error de conexión: {e}")

# Función para ejecutar get_users() cada 25 minutos
def repeat_get_users():
    while True:
        get_users()
        time.sleep(600)  # Espera 10 minutos

# Inicia un hilo para ejecutar repeat_get_users() en segundo plano
user_thread = threading.Thread(target=repeat_get_users)
user_thread.start()

if colabconfig["server_type"] == "forge":

    version = colabconfig["server_version"]

    if colabconfig["server_version"] < "1.17":

        oldpathtoforge = glob.glob(f"/workspaces/codespaces-jupyter/Minecraft-server/forge-{version}-*.jar")

        if oldpathtoforge: # Checa si la lista no esta vacia

            path = oldpathtoforge[0] # Obtiene la primera ruta de la lista

            print(path)

            !java $memory_allocation -jar "{path}" nogui

        else:

            print("No se encontro la version de Forge universal.")

    else:

        pathtoforge = glob.glob(f"/workspaces/codespaces-jupyter/Minecraft-server/libraries/net/minecraftforge/forge/{version}-/unix_.txt")

        if pathtoforge: # Checa si la lista no esta vacia

            path = pathtoforge[0] # Obtiene la primera ruta de la lista

            print(path)

            !java @user_jvm_args.txt "@{path}" "$@"

        else:

            print("No unix_args.txt found.")
else:

    !java $memory_allocation $server_flags -jar $jar_name nogui



# Ciclo para realizar las solicitudes a JSONPlaceholder cada minuto

while True:
    time.sleep(60)

apt cache successfully updated
banned-ips.json      defaultconfigs  mohist-config    server.properties
banned-players.json  eula.txt	     mohist.jar       spigot.yml
bukkit.yml	     help.yml	     ops.json	      usercache.json
colabconfig.json     journeymap      patchouli_books  usernamecache.json
commands.yml	     libraries	     permissions.yml  whitelist.json
config		     logs	     plugins	      world
crash-reports	     mods	     server-icon.png
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `sudo apt-get install openjdk-17-jre-headless &>/dev/null and print("OpenJDK 17 installed") or print("Failed to install OpenJDK 17.")'
You are using a downgraded java version. Minecraft 1.17+ might not work.
Usando ngrok
Consigue tu authtoken de https://dashboard.ngrok.com/auth
Authtoken saved to configuration file: /home/codespace/.config/ngrok/ngrok.yml
La IP de tu servidor es 6.tcp.ngrok.io:15603
Iniciando servidor...
Lista de usuarios:
Nombre: Leanne Grah

t=2023-12-20T02:34:14+0000 lvl=warn msg="failed to open private leg" id=1ab7157b4ed3 typ=proxy privaddr=localhost:25565 err="dial tcp [::1]:25565: connect: connection refused"


All libraries are nice :)!
Searching incompatible mods...


t=2023-12-20T02:34:16+0000 lvl=warn msg="failed to open private leg" id=662bd0c95b6f typ=proxy privaddr=localhost:25565 err="dial tcp [::1]:25565: connect: connection refused"
t=2023-12-20T02:34:16+0000 lvl=warn msg="failed to open private leg" id=bfa137d773f8 typ=proxy privaddr=localhost:25565 err="dial tcp [::1]:25565: connect: connection refused"
t=2023-12-20T02:34:19+0000 lvl=warn msg="failed to open private leg" id=ab59769cbdda typ=proxy privaddr=localhost:25565 err="dial tcp [::1]:25565: connect: connection refused"


[02:34:20 INFO]: ModLauncher running: args [--launchTarget, forgeserver, --fml.forgeVersion, 47.2.17, --fml.mcVersion, 1.20.1, --fml.forgeGroup, net.minecraftforge, --fml.mcpVersion, 20230612.114412, nogui]
[02:34:20 INFO]: ModLauncher 10.0.9+10.0.9+main.dcd20f30 starting: java version 21.0.1 by Microsoft; OS Linux arch amd64 version 6.2.0-1018-azure


...

[02:34:55 INFO]: Adding datapack listener capsid_recipes
[02:34:55 INFO]: Adding datapack listener altar_of_amethyst_recipes


t=2023-12-20T02:34:56+0000 lvl=warn msg="failed to open private leg" id=80503e4d9081 typ=proxy privaddr=localhost:25565 err="dial tcp [::1]:25565: connect: connection refused"


[02:34:56 INFO]: Skipping loading recipe supplementaries:inspirations/blackboard_clear as it's serializer returned null
[02:34:56 INFO]: Skipping loading recipe twilightforest:jeed/frosted as it's serializer returned null
[02:34:56 INFO]: Skipping loading recipe supplementaries:inspirations/flag_dye as it's serializer returned null
[02:34:56 INFO]: Skipping loading recipe twilightforest:jeed/hunger as it's serializer returned null
[02:34:56 ERROR]: Parsing error loading recipe magistuarmory:giant_plume
com.google.gson.JsonSyntaxException: Unknown item 'magistuarmory:giant_plume'
	at net.minecraftforge.common.crafting.CraftingHelper.getItem(CraftingHelper.java:160) ~[forge-1.20.1-47.2.17-universal.jar%23357!/:?]
	at net.minecraftforge.common.crafting.CraftingHelper.getItemStack(CraftingHelper.java:186) ~[forge-1.20.1-47.2.17-universal.jar%23357!/:?]
	at net.minecraft.world.item.crafting.ShapedRecipe.m_151274_(ShapedRecipe.java:355) ~[server-1.20.1-20230612.114412-srg.jar%23352!/:?]
	at 

t=2023-12-20T02:34:59+0000 lvl=warn msg="failed to open private leg" id=0de361b5df03 typ=proxy privaddr=localhost:25565 err="dial tcp [::1]:25565: connect: connection refused"
t=2023-12-20T02:35:01+0000 lvl=warn msg="failed to open private leg" id=8a154949ad6b typ=proxy privaddr=localhost:25565 err="dial tcp [::1]:25565: connect: connection refused"


[02:35:01 INFO]: Finished additional setup in 50 ms
[02:35:01 INFO]: Starting minecraft server version 1.20.1
[02:35:01 INFO]: Loading properties
[02:35:01 INFO]: Default game type: SURVIVAL
[02:35:01 INFO]: Generating keypair
[02:35:01 INFO]: Starting Minecraft server on *:25565
[02:35:01 INFO]: Using epoll channel type
[02:35:02 WARN]: **** SERVER IS RUNNING IN OFFLINE/INSECURE MODE!
[02:35:02 WARN]: The server will make no attempt to authenticate usernames. Beware.
[02:35:02 WARN]: While this makes the game possible to play without internet access, it also opens up the ability for hackers to connect with any username they choose.
[02:35:02 WARN]: To change this, set "online-mode" to "true" in the server.properties file.
[02:35:02 INFO]: This server is running Mohist version 1.20.1-535 (MC: 1.20.1) (Implementing API version 1.20.1-R0.1-SNAPSHOT, Forge version 47.2.17, NeoForge version 47.2.17)
[02:35:04 INFO]: Debug logging is enabled
[02:35:04 INFO]: Server Ping Player Sample Count: